In [1]:
import pandas as pd
import numpy as np
import requests
import json
import urllib
import datetime

pd.set_option("display.max_columns", 300)
pd.set_option('display.max_rows', 300)
pd.options.mode.chained_assignment = None

Load in `df_bq`, which contains results from [this query](https://bigquery.cloud.google.com/results/infusionsoft-data-production:US.bquijob_47811bc0_16253f5e66f) from our payment records, and `df_wp`, which contains an export of WePay CAD transaction data.

Then, limit `df_bq` and `df_wp` to payments in January and February, filter by app name `qv200`, and limit `df_bq` to `PaymentSubType = 'INF_PAY'`.

In [2]:
df_bq = pd.read_csv('results-20180323-104407.csv')
df_bq['day_created'] = pd.to_datetime(df_bq.DateCreated)
df_bq['day_created'] = pd.DatetimeIndex(df_bq['day_created'] ).normalize()

df_bq = df_bq[(df_bq.appName == 'qv200') & (df_bq.day_created >= '2018-01-01') & (df_bq.day_created < '2018-03-01') & (df_bq.PaymentSubType == 'INF_PAY')].reset_index()

df_bq

,index,Id,ContactId,PaymentId,TotalAmountPaid,PaymentDate,TransactionType,CollectionMethod,PaymentSubType,PaymentGatewayId,ErrorMessage,TestPayment,GeneralValidationPassed,StrategyValidationPassed,ExecuteRan,ResultsBuilt,ErrorsMapped,Source,Deleted,DateCreated,LastUpdated,appName,PayType,CurrencyCode,day_created
0,5,114251,14061,3505,226.000000,2018-01-16 16:51:27.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-16 16:51:32.000000 UTC,2018-01-16 16:51:32.000000 UTC,qv200,Credit Card,CAD,2018-01-16
1,7,112751,7711,3457,110.680000,2018-01-07 04:38:51.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-07 04:38:54.000000 UTC,2018-01-07 04:38:54.000000 UTC,qv200,Credit Card,CAD,2018-01-07
2,8,112749,7711,3455,33.840000,2018-01-07 04:28:17.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-07 04:28:21.000000 UTC,2018-01-07 04:28:21.000000 UTC,qv200,Credit Card,CAD,2018-01-07
3,14,113517,12315,3467,33.840000,2018-01-11 23:16:38.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-11 23:16:41.000000 UTC,2018-01-11 23:16:41.000000 UTC,qv200,Credit Card,CAD,2018-01-11
4,15,113519,20317,3469,67.690002,2018-01-11 23:52:59.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-11 23:53:04.000000 UTC,2018-01-11 23:53:04.000000 UTC,qv200,Credit Card,CAD,2018-01-11
5,17,120431,22689,3655,75.709999,2018-02-23 02:08:36.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-02-23 02:08:40.000000 UTC,2018-02-23 02:08:40.000000 UTC,qv200,Credit Card,CAD,2018-02-23
6,20,115935,12473,3565,110.680000,2018-01-26 04:08:31.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,autochargeTask,False,2018-01-26 04:08:35.000000 UTC,2018-01-26 04:08:35.000000 UTC,qv200,Credit Card,CAD,2018-01-26
7,21,115825,21519,3561,33.840000,2018-01-25 15:07:51.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-25 15:07:56.000000 UTC,2018-01-25 15:07:56.000000 UTC,qv200,Credit Card,CAD,2018-01-25
8,22,111929,3985,3433,110.680000,2018-01-01 18:56:22.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-01 18:56:27.000000 UTC,2018-01-01 18:56:27.000000 UTC,qv200,Credit Card,CAD,2018-01-01
9,23,120433,22055,3657,75.709999,2018-02-23 05:26:46.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-02-23 05:26:52.000000 UTC,2018-02-23 05:26:52.000000 UTC,qv200,Credit Card,CAD,2018-02-23


In [3]:
df_wp = pd.read_csv('wepay.csv')
df_wp['create_time'] = pd.to_datetime(df_wp.create_time, unit='s')
df_wp['day_created'] = pd.to_datetime(df_wp['create_time'])
df_wp['day_created'] = pd.DatetimeIndex(df_wp['day_created'] ).normalize()

df_wp = df_wp[df_wp.account_name.str.contains('qv200') & (df_wp.day_created >= '2018-01-01') & (df_wp.day_created < '2018-03-01')].reset_index()

df_wp.drop(['account_name','checkout_uri','payer_email','payer_name'], axis=1, inplace=True)

df_wp

,index,checkout_id,account_id,type,short_description,currency,amount,fee_payer,state,redirect_uri,auto_capture,app_fee,create_time,amount_refunded,amount_charged_back,gross,fee,long_description,reference_id,callback_uri,shipping_address,shipping_fee,preapproval_id,tax,day_created
0,1112,1854219689,479437278,GOODS,Auto Charge For Invoice Ids:3377,CAD,339.00,payee,captured,NaN,1,0,2018-02-28 20:10:29,0.00,0.0,339.00,10.13,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-28
1,1131,1442245989,479437278,GOODS,Auto Charge For Invoice Ids:3375,CAD,197.75,payee,refunded,NaN,1,0,2018-02-28 14:39:41,197.75,0.0,197.75,6.03,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-28
2,1244,693674592,479437278,GOODS,Auto Charge For Invoice Ids:3367,CAD,565.00,payee,captured,NaN,1,0,2018-02-26 14:35:55,0.00,0.0,565.00,16.68,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-26
3,1282,1624981774,479437278,GOODS,Auto Charge For Invoice Ids:3365,CAD,197.75,payee,captured,NaN,1,0,2018-02-25 14:27:40,0.00,0.0,197.75,6.03,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-25
4,1294,453644247,479437278,GOODS,Auto Charge For Invoice Ids:3363,CAD,339.00,payee,captured,NaN,1,0,2018-02-24 19:18:17,0.00,0.0,339.00,10.13,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-24
5,1328,353088085,479437278,GOODS,Auto Charge For Invoice Ids:3361,CAD,75.71,payee,captured,NaN,1,0,2018-02-23 05:26:49,0.00,0.0,75.71,2.49,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-23
6,1343,1207757106,479437278,GOODS,Auto Charge For Invoice Ids:3359,CAD,75.71,payee,captured,NaN,1,0,2018-02-23 02:08:38,0.00,0.0,75.71,2.49,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-23
7,1392,456973385,479437278,GOODS,Auto Charge For Invoice Ids:3357,CAD,339.00,payee,captured,NaN,1,0,2018-02-21 20:25:58,0.00,0.0,339.00,10.13,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-21
8,1394,2015559781,479437278,GOODS,Auto Charge For Invoice Ids:3355,CAD,565.00,payee,captured,NaN,1,0,2018-02-21 19:37:55,0.00,0.0,565.00,16.68,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-21
9,1435,1624983822,479437278,GOODS,Auto Charge For Invoice Ids:3351,CAD,339.00,payee,captured,NaN,1,0,2018-02-20 18:50:13,0.00,0.0,339.00,10.13,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-20


The lengths of these two identically-filtered datasets are given:

In [4]:
print(f'Our BQ data is {len(df_bq)} rows')
print(f'The WePay data is {len(df_wp)} rows')

Our BQ data is 64 rows
The WePay data is 61 rows


Group these two by day into `wp_groupdays` and `bq_groupdays`, and then find for which days the transaction counts are different.

In [5]:
wp_groupdays = df_wp.groupby('day_created').agg('count')['amount'].reset_index()
bq_groupdays = df_bq.groupby('day_created').agg('count')['TotalAmountPaid'].reset_index()

days_unmatched = []
for x in range(0,len(bq_groupdays)):
    if bq_groupdays.iloc[x,1] != wp_groupdays.iloc[x,1]:
        print(bq_groupdays.iloc[x,0])
        print(f'bq says we have {bq_groupdays.iloc[x,1]} transactions on this date, while wp says we have {wp_groupdays.iloc[x,1]}')
        days_unmatched.append(datetime.datetime.strftime(bq_groupdays.iloc[x,0], format='%Y-%m-%d'))

2018-01-28 00:00:00
bq says we have 2 transactions on this date, while wp says we have 1
2018-02-18 00:00:00
bq says we have 4 transactions on this date, while wp says we have 3
2018-02-28 00:00:00
bq says we have 3 transactions on this date, while wp says we have 2


Loop through these `unmatched_days` and print off what we have.

In [6]:
df_bq[df_bq.day_created.isin(days_unmatched)].sort_values('PaymentDate')

,index,Id,ContactId,PaymentId,TotalAmountPaid,PaymentDate,TransactionType,CollectionMethod,PaymentSubType,PaymentGatewayId,ErrorMessage,TestPayment,GeneralValidationPassed,StrategyValidationPassed,ExecuteRan,ResultsBuilt,ErrorsMapped,Source,Deleted,DateCreated,LastUpdated,appName,PayType,CurrencyCode,day_created
31,65,116301,7711,3573,33.840000,2018-01-28 15:34:34.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-01-28 15:34:37.000000 UTC,2018-01-28 15:34:37.000000 UTC,qv200,Credit Card,CAD,2018-01-28
28,62,116305,7711,3575,33.840000,2018-01-28 15:35:42.000000 UTC,REFUND,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,NaN,False,2018-01-28 15:35:42.000000 UTC,2018-01-28 15:35:42.000000 UTC,qv200,Refund,NaN,2018-01-28
38,78,119673,6018,3633,75.709999,2018-02-18 14:12:23.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-02-18 14:12:28.000000 UTC,2018-02-18 14:12:28.000000 UTC,qv200,Credit Card,CAD,2018-02-18
37,77,119675,22589,3635,339.000000,2018-02-18 14:50:58.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-02-18 14:51:03.000000 UTC,2018-02-18 14:51:03.000000 UTC,qv200,Credit Card,CAD,2018-02-18
35,75,119715,20159,3637,339.000000,2018-02-18 16:20:43.000000 UTC,REFUND,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,NaN,False,2018-02-18 16:20:45.000000 UTC,2018-02-18 16:20:45.000000 UTC,qv200,Refund,NaN,2018-02-18
36,76,119757,14225,3641,339.000000,2018-02-18 23:19:08.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-02-18 23:19:11.000000 UTC,2018-02-18 23:19:11.000000 UTC,qv200,Credit Card,CAD,2018-02-18
63,121,121285,20405,3669,197.750000,2018-02-28 14:39:39.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-02-28 14:39:44.000000 UTC,2018-02-28 14:39:44.000000 UTC,qv200,Credit Card,CAD,2018-02-28
62,120,121327,23343,3671,339.000000,2018-02-28 20:10:26.000000 UTC,CAPTURE,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,orderForm,False,2018-02-28 20:10:32.000000 UTC,2018-02-28 20:10:32.000000 UTC,qv200,Credit Card,CAD,2018-02-28
61,119,121329,20405,3673,197.750000,2018-02-28 20:14:25.000000 UTC,REFUND,PAYMENT_GATEWAY,INF_PAY,1,NaN,False,True,True,True,True,False,NaN,False,2018-02-28 20:14:27.000000 UTC,2018-02-28 20:14:27.000000 UTC,qv200,Refund,NaN,2018-02-28


In [7]:
df_wp[df_wp.day_created.isin(days_unmatched)].sort_values('create_time')

,index,checkout_id,account_id,type,short_description,currency,amount,fee_payer,state,redirect_uri,auto_capture,app_fee,create_time,amount_refunded,amount_charged_back,gross,fee,long_description,reference_id,callback_uri,shipping_address,shipping_fee,preapproval_id,tax,day_created
28,2378,1969011946,479437278,GOODS,Auto Charge For Invoice Ids:3283,CAD,33.84,payee,refunded,NaN,1,0,2018-01-28 15:34:34,33.84,0.0,33.84,1.28,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-01-28
13,1508,1500331886,479437278,GOODS,Auto Charge For Invoice Ids:3343,CAD,75.71,payee,captured,NaN,1,0,2018-02-18 14:12:25,0.00,0.0,75.71,2.49,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-18
12,1507,1887684556,479437278,GOODS,Auto Charge For Invoice Ids:3345,CAD,339.00,payee,captured,NaN,1,0,2018-02-18 14:51:00,0.00,0.0,339.00,10.13,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-18
11,1502,642048812,479437278,GOODS,Auto Charge For Invoice Ids:3347,CAD,339.00,payee,captured,NaN,1,0,2018-02-18 23:19:08,0.00,0.0,339.00,10.13,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-18
1,1131,1442245989,479437278,GOODS,Auto Charge For Invoice Ids:3375,CAD,197.75,payee,refunded,NaN,1,0,2018-02-28 14:39:41,197.75,0.0,197.75,6.03,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-28
0,1112,1854219689,479437278,GOODS,Auto Charge For Invoice Ids:3377,CAD,339.00,payee,captured,NaN,1,0,2018-02-28 20:10:29,0.00,0.0,339.00,10.13,NaN,NaN,https://qv200.infusionsoft.com/app/ipn/wepay/t...,NaN,0,NaN,0,2018-02-28


We can now see the three entries in BQ that are not in WePay - the CAPTURE for 33.84 on 2018-01-28, the REFUND for 339.00 on 2018-02-18, and the CAPTURE for 197.75 on 2018-02-28. 